In [1]:
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt


In [16]:
import pandas as pd
import numpy as np

# 1. 读取CSV文件
df = pd.read_csv(r"C:\Users\rog\Desktop\CPI_X13ARIMA_SEATS.csv")

# 2. 提取"China"列并计算对数环比增长率
df['China_log_growth_rate'] = np.log(df['China'] / df['China'].shift(1))

# 3. 计算描述性统计
china_desc_stats = df['China_log_growth_rate'].describe()

# 4. 显示结果
print(china_desc_stats)



count    312.000000
mean       0.001558
std        0.006206
min       -0.013986
25%       -0.002002
50%        0.000997
75%        0.004989
max        0.025668
Name: China_log_growth_rate, dtype: float64


In [17]:
import pandas as pd
from statsmodels.tsa.stattools import adfuller

# 1. 读取CSV文件
df = pd.read_csv(r"C:\Users\rog\Desktop\CPI_X13ARIMA_SEATS.csv")

# 2. 定义一个函数进行ADF检验
def adf_test(series):
    result = adfuller(series.dropna())  # ADF检验，dropna是为了避免缺失值
    return result[1]  # 返回p值

# 3. 对每个国家的CPI数据进行ADF检验
non_stationary_countries = []

# 4. 检验每列数据
for column in df.columns[1:]:  # 假设第一列是Date，跳过它
    p_value = adf_test(df[column])
    if p_value > 0.05:  # p值大于0.05时，认为数据不平稳
        non_stationary_countries.append(column)

# 5. 输出不平稳的国家
print("不平稳的国家：")
print(non_stationary_countries)


不平稳的国家：
['Afghanistan', 'Albania', 'Algeria', 'Angola', 'Antigua and Barbuda', 'Armenia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Benin', 'Bolivia', 'Bosnia and Herzegovina', 'Botswana', 'Brazil', 'Brunei', 'Bulgaria', 'Burundi', 'Burkina Faso', 'Cape Verde', 'Cambodia', 'Cameroon', 'Canada', 'Central African Republic', 'Chad', 'Chile', 'China', 'Colombia', 'Congo', 'Costa Rica', 'Croatia', 'Cyprus', 'Czech Republic', 'Denmark', 'Djibouti', 'Dominica', 'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador', 'Equatorial Guinea', 'Estonia', 'Swaziland', 'Ethiopia', 'Fiji', 'Finland', 'France', 'Gabon', 'Gambia', 'Georgia', 'Germany', 'Ghana', 'Greece', 'Grenada', 'Guatemala', 'Guinea', 'Guinea-Bissau', 'Guyana', 'Haiti', 'Honduras', 'Hong Kong Special Administrative Region of China', 'Hungary', 'Iceland', 'India', 'Indonesia', 'Iran', 'Iraq', 'Ireland', 'Israel', 'Italy', "Côte d'Ivoire", 'Jamaica', 'Japan', 'Jordan', 'Kazakhstan',

In [19]:
import pandas as pd
from statsmodels.tsa.seasonal import STL
from statsmodels.tsa.stattools import adfuller

# 1. 读取CSV文件
df = pd.read_csv(r"C:\Users\rog\Desktop\CPI_X13ARIMA_SEATS.csv")

# 2. 定义一个函数进行ADF检验
def adf_test(series):
    # 去掉缺失值
    series = series.dropna()
    if len(series) == 0:  # 如果数据为空，则返回NaN
        return float('nan')
    result = adfuller(series)  # ADF检验
    return result[1]  # 返回p值

# 3. 季节性调整（STL分解）并检查每个国家的CPI数据
def seasonally_adjust(df, column):
    cpi_series = df[column].dropna()  # 去掉缺失值
    
    if len(cpi_series) == 0:  # 如果数据为空，则跳过
        return float('nan'), cpi_series
    
    # 使用STL进行季节性调整，周期为12（月度数据通常选择12作为季节性周期）
    stl = STL(cpi_series, period=12)
    result = stl.fit()
    
    # 获取季节性调整后的残差部分
    adjusted_cpi = result.resid
    
    # 进行ADF检验
    p_value = adf_test(adjusted_cpi)
    return p_value, adjusted_cpi

# 存储季节性调整后通过平稳性检验的国家
stationary_countries = []

# 4. 对每个国家进行季节性调整并检查是否平稳
for column in df.columns[1:]:  # 假设第一列是Date，跳过它
    p_value, adjusted_cpi = seasonally_adjust(df, column)
    
    # 跳过p值为NaN的列
    if p_value != p_value:  # 检查p_value是否为NaN
        continue
    
    if p_value < 0.05:  # 如果p值小于0.05，说明调整后的数据是平稳的
        stationary_countries.append(column)

# 5. 输出季节性调整后平稳的国家
print("季节性调整后平稳的国家：")
print(stationary_countries)



季节性调整后平稳的国家：
['Afghanistan', 'Albania', 'Algeria', 'Angola', 'Antigua and Barbuda', 'Armenia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Benin', 'Bolivia', 'Bosnia and Herzegovina', 'Botswana', 'Brazil', 'Brunei', 'Bulgaria', 'Burundi', 'Burkina Faso', 'Cape Verde', 'Cambodia', 'Cameroon', 'Canada', 'Central African Republic', 'Chad', 'Chile', 'China', 'Colombia', 'Congo', 'Costa Rica', 'Croatia', 'Cyprus', 'Czech Republic', 'Denmark', 'Djibouti', 'Dominica', 'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador', 'Equatorial Guinea', 'Estonia', 'Swaziland', 'Ethiopia', 'Fiji', 'Finland', 'France', 'Gabon', 'Gambia', 'Georgia', 'Germany', 'Ghana', 'Greece', 'Grenada', 'Guatemala', 'Guinea', 'Guinea-Bissau', 'Guyana', 'Haiti', 'Honduras', 'Hong Kong Special Administrative Region of China', 'Hungary', 'Iceland', 'India', 'Indonesia', 'Iran', 'Iraq', 'Ireland', 'Israel', 'Italy', "Côte d'Ivoire", 'Jamaica', 'Japan', 'Jordan', 'Kazakhs

In [20]:
import pandas as pd
from statsmodels.tsa.seasonal import STL
from statsmodels.tsa.stattools import adfuller

# 1. 读取CSV文件
df = pd.read_csv(r"C:\Users\rog\Desktop\CPI_X13ARIMA_SEATS.csv")

# 2. 定义一个函数进行ADF检验
def adf_test(series):
    series = series.dropna()
    if len(series) == 0:  # 如果数据为空，则返回NaN
        return float('nan')
    result = adfuller(series)  # ADF检验
    return result[1]  # 返回p值

# 3. 季节性调整（STL分解）并检查每个国家的CPI数据
def seasonally_adjust(df, column):
    cpi_series = df[column].dropna()  # 去掉缺失值
    
    if len(cpi_series) == 0:  # 如果数据为空，则跳过
        return float('nan'), cpi_series
    
    # 使用STL进行季节性调整，周期为12（月度数据通常选择12作为季节性周期）
    stl = STL(cpi_series, period=12)
    result = stl.fit()
    
    # 获取季节性调整后的残差部分
    adjusted_cpi = result.resid
    
    # 进行ADF检验
    p_value = adf_test(adjusted_cpi)
    return p_value, adjusted_cpi

# 存储季节性调整后通过平稳性检验的国家
stationary_countries = []

# 季节性调整每个国家的CPI并进行ADF检验
for column in df.columns[1:]:  # 假设第一列是Date，跳过它
    p_value, adjusted_cpi = seasonally_adjust(df, column)
    
    if p_value != p_value:  # 如果p_value是NaN，则跳过
        continue
    
    if p_value < 0.05:  # 如果p值小于0.05，说明调整后的数据是平稳的
        stationary_countries.append(column)

# 输出季节性调整后平稳的国家
print("季节性调整后平稳的国家：")
print(stationary_countries)

# 4. 输出中国、美国、法国的描述性统计
countries_to_analyze = ['China', 'USA', 'France']

for country in countries_to_analyze:
    if country in df.columns:
        print(f"\n{country} 描述性统计：")
        # 获取季节性调整后的数据
        p_value, adjusted_cpi = seasonally_adjust(df, country)
        
        # 输出描述性统计
        print(adjusted_cpi.describe())


季节性调整后平稳的国家：
['Afghanistan', 'Albania', 'Algeria', 'Angola', 'Antigua and Barbuda', 'Armenia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Benin', 'Bolivia', 'Bosnia and Herzegovina', 'Botswana', 'Brazil', 'Brunei', 'Bulgaria', 'Burundi', 'Burkina Faso', 'Cape Verde', 'Cambodia', 'Cameroon', 'Canada', 'Central African Republic', 'Chad', 'Chile', 'China', 'Colombia', 'Congo', 'Costa Rica', 'Croatia', 'Cyprus', 'Czech Republic', 'Denmark', 'Djibouti', 'Dominica', 'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador', 'Equatorial Guinea', 'Estonia', 'Swaziland', 'Ethiopia', 'Fiji', 'Finland', 'France', 'Gabon', 'Gambia', 'Georgia', 'Germany', 'Ghana', 'Greece', 'Grenada', 'Guatemala', 'Guinea', 'Guinea-Bissau', 'Guyana', 'Haiti', 'Honduras', 'Hong Kong Special Administrative Region of China', 'Hungary', 'Iceland', 'India', 'Indonesia', 'Iran', 'Iraq', 'Ireland', 'Israel', 'Italy', "Côte d'Ivoire", 'Jamaica', 'Japan', 'Jordan', 'Kazakhs